In [1]:
import uuid
from typing import Dict
from datetime import datetime
from pydantic import BaseModel
from langchain.memory import ConversationBufferMemory
from langchain.schema import messages_from_dict, messages_to_dict
from groq import Groq
import os
from dotenv import load_dotenv

load_dotenv()

# groq llm client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

In [2]:
def groq_chat(messages: list) -> str:

    # if SessionId.session_id not in session_state:
    
    # Create a simple but context-aware prompt
    prompt = f"Answer the following user query clearly and concisely:\n\n{messages}"

    # Send request to Groq model
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    # Extract and return response text
    return response.choices[0].message.content.strip()

In [3]:
groq_chat(messages=["what is the capital of India?"])

'The capital of India is New Delhi.'

In [4]:
class ChatRequest(BaseModel):
    query: str

class SessionId(BaseModel):
    session_id: str = None  # Optional, if not provided, create new

class ChatResponse(BaseModel):
    response: str

In [5]:
# --- Memory Management ---
# In-memory active sessions
active_sessions: Dict[str, ConversationBufferMemory] = {}

In [6]:
active_sessions

{}

In [7]:
session_id = "a33ba0aa256c48bea03793e70b733113,"


if session_id in active_sessions.values():
    print(session_id)
else:
    print("created sessison id")

created sessison id


In [8]:
active_sessions

{}

In [9]:
# filter session id
def filter_session_id(state: SessionId) -> SessionId:
    
    if state.session_id in active_sessions.values():
        # Create new session
        state['session_id'] = state.sesstion_id
        
    return state

filter_session_id(SessionId(session_id="fda9441bb78e46a79398ff0095cc8593"))

SessionId(session_id='fda9441bb78e46a79398ff0095cc8593')

In [10]:
# --- Session filter/create function ---
def filter_or_create_session_id(state: SessionId) -> str:
    """
    Ensures session_id exists in active_sessions.
    If not, creates a new one with ConversationBufferMemory.
    """
    session_id = state.session_id

    if not session_id or session_id not in active_sessions:
        session_id = str(uuid.uuid4().hex)
        active_sessions[session_id] = ConversationBufferMemory(
            memory_key="chat_history",
            return_messages=True
        )
        print(f"🆕 Created new session: {session_id}")
    else:
        print(f"✅ Using existing session: {session_id}")

    return session_id

In [32]:
def filter_or_create_session_id(session_id: str) -> str:

    if session_id not in active_sessions:
        # Create new session
        session_id = str(uuid.uuid4().hex)
        active_sessions[session_id] = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
    else:
        # Use existing session
        return session_id
    
# filter or create sesison id
filtered_session_id = filter_or_create_session_id(session_id="fda9441bb78e46a79398ff0095cc8593")
print(filtered_session_id)

None


In [30]:
# ============ filter_or_create_session_id
# def filter_or_create_session_id(state: SessionId):
    
#     session_id = state.session_id

#     if session_id not in active_sessions:
#         # Create new session
#         state.session_id = str(uuid.uuid4().hex)
#         active_sessions[state.session_id] = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
#     else:
#         # Use existing session
#         return state.session_id
    
# # filter or create sesison id
# filtered_session_id = filter_or_create_session_id(SessionId(session_id="fda9441bb78e46a79398ff0095cc8593"))
# print(filtered_session_id)

In [12]:
active_sessions

{'1d15849f245a4e378421430c4e7ec532': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history')}

In [13]:
session_id = "fda9441bb78e46a79398ff0095cc8593"

if filter_or_create_session_id(SessionId(session_id=session_id)):
    print(session_id)
else:
    print("created session id")

created session id


In [24]:
def chatbot(query: str) -> str:
    
    # Create a simple but context-aware prompt
    prompt = f"Answer the following user query clearly and concisely:\n\n{query}"

    # Send request to Groq model
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    # Extract and return response text
    return response.choices[0].message.content.strip()

In [25]:
response = chatbot(query="what is the capital of India?")
print(response)

The capital of India is New Delhi.


<!-- ### Now manage the session with memeory in chatbot -->

In [16]:
active_sessions

{'1d15849f245a4e378421430c4e7ec532': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history'),
 '5c65fbcfcd324943a2eee0a19cc0670e': ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[]), return_messages=True, memory_key='chat_history')}

In [17]:
session_id = "fda9441bb78e46a79398ff0095cc8593"

if filter_or_create_session_id(SessionId(session_id=session_id)):
    print(session_id)
else:
    print("created session id")

created session id


In [18]:
query: ChatRequest = "hello"

In [19]:
query

'hello'

In [38]:
class ChatRequest(BaseModel):
    query: str
    session_id: str = None  # Optional, if not provided, create new

class ChatResponse(BaseModel):
    session_id: str = None  # Return session_id in response
    history: list = []  # Chat history

In [42]:
# --- Chatbot function with conversation memory ---
def chat_with_memory(state: ChatRequest) -> ChatResponse:
    """
    Manage conversational chat sessions with memory.
    Creates or reuses a session and keeps conversation context.
    """
    # 1. Ensure session exists
    session_id = filter_or_create_session_id(session_id=state.session_id)
    memory = active_sessions[session_id]

    # 2. Add user message to memory
    memory.chat_memory.add_user_message(state.query)

    # 3. Convert chat memory to Groq messages format
    messages_for_groq = [
        {"role": "user" if msg.type == "human" else "assistant", "content": msg.content}
        for msg in memory.chat_memory.messages]

    # 4. Get response from Groq model
    assistant_reply = chatbot(query=messages_for_groq)

    # 5. Add assistant reply to memory
    memory.chat_memory.add_ai_message(assistant_reply)

    # 6. Prepare history for response
    chat_history = [
        {"role": "user" if msg.type == "human" else "assistant", "content": msg.content}
        for msg in memory.chat_memory.messages]
    
    # 7. Return response
    return ChatResponse(session_id=session_id, history=chat_history)

In [49]:
user_input = ChatRequest(query="What is the capital of India?")
user_input

ChatRequest(query='What is the capital of India?', session_id=None)

In [ ]:
chat_with_memory

In [22]:
def get_response(query) -> str:
    response = chatbot(ChatRequest(query=query)).response
    return response

In [23]:
get_response(query="what is the capital of India?")

'The capital of India is New Delhi.'